In [ ]:
!pip install texthero -q

     |████████████████████████████████| 245kB 6.8MB/s 
     |████████████████████████████████| 1.4MB 11.4MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# TensorFlow
import tensorflow as tf
import tensorflow_hub as hub

#Library for Text Processing
#import texthero as hero

#Sk Learn Library
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Garbage
import gc

#Warnings
import warnings
warnings.filterwarnings("ignore")

#Tabulate
from tabulate import tabulate

In [ ]:
raw_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Copy of labeled_data.csv', header='infer')

In [ ]:
raw_data

,index,language,usage.text_units,usage.text_characters,usage.features,sentiment.document.score,sentiment.document.label,sentiment.targets.mixed,sentiment.targets.label,sentiment.targets.score,sentiment.targets.text
0,0,en,1,94,1,0.000000,neutral,NaN,neutral,0.000000,Ollie's Bargain Outlet's (OLLI) CEO John Swyge...
1,1,en,1,69,1,0.000000,neutral,NaN,neutral,0.000000,"Mar. 18, 2021 11:24 PM ETOllie's Bargain Outle..."
2,2,en,1,31,1,0.673492,positive,NaN,positive,0.673492,Q4: 2021-03-18 Earnings Summary
3,3,en,1,79,1,-0.533729,negative,NaN,negative,-0.533729,EPS of $0.97 beats by $0.12 | Revenue of $515....
4,4,en,1,110,1,0.000000,neutral,NaN,neutral,0.000000,"Ollie's Bargain Outlet Holdings, Inc. (NASDAQ:..."
...,...,...,...,...,...,...,...,...,...,...,...
4392,4392,en,1,353,1,0.753839,positive,NaN,positive,0.774738,So now let's go to the slide. So this gives yo...
4393,4393,en,1,81,1,0.862047,positive,NaN,positive,0.862047,"And more importantly, 65% of our total sales a..."
4394,4394,en,1,380,1,0.538452,positive,1.0,positive,0.421919,Our exclusive sales participation reached 35% ...
4395,4395,en,1,252,1,0.488623,positive,NaN,positive,0.640116,Because our data analysis shows that an averag...


In [ ]:
pip install flat-table

In [ ]:
import flat_table
import json
import pandas as pd
from pandas.io.json import json_normalize

data = json.load(open("/content/drive/MyDrive/DWBI/out.json"))
df = json_normalize(data)
table = flat_table.normalize(df)
table.to_csv("/content/drive/MyDrive/out_data.csv", index=False)

In [ ]:
data = raw_data[['sentiment.targets.text','sentiment.targets.label']]

In [ ]:
data

,sentiment.targets.text,sentiment.targets.label
0,Ollie's Bargain Outlet's (OLLI) CEO John Swyge...,neutral
1,"Mar. 18, 2021 11:24 PM ETOllie's Bargain Outle...",neutral
2,Q4: 2021-03-18 Earnings Summary,positive
3,EPS of $0.97 beats by $0.12 | Revenue of $515....,negative
4,"Ollie's Bargain Outlet Holdings, Inc. (NASDAQ:...",neutral
...,...,...
4392,So now let's go to the slide. So this gives yo...,positive
4393,"And more importantly, 65% of our total sales a...",positive
4394,Our exclusive sales participation reached 35% ...,positive
4395,Because our data analysis shows that an averag...,positive


In [ ]:
data = data.rename(columns={'sentiment.targets.text': 'texts', 'sentiment.targets.label': 'label'})

In [ ]:
data

,texts,label
0,Ollie's Bargain Outlet's (OLLI) CEO John Swyge...,neutral
1,"Mar. 18, 2021 11:24 PM ETOllie's Bargain Outle...",neutral
2,Q4: 2021-03-18 Earnings Summary,positive
3,EPS of $0.97 beats by $0.12 | Revenue of $515....,negative
4,"Ollie's Bargain Outlet Holdings, Inc. (NASDAQ:...",neutral
...,...,...
4392,So now let's go to the slide. So this gives yo...,positive
4393,"And more importantly, 65% of our total sales a...",positive
4394,Our exclusive sales participation reached 35% ...,positive
4395,Because our data analysis shows that an averag...,positive


In [ ]:
data.label.value_counts()

positive    2313
neutral     1732
negative     352
Name: label, dtype: int64

In [ ]:
data = data[data.label != 'neutral']

In [ ]:
data=data.reset_index(drop=True)

In [ ]:
#Encode the Label to convert it into numerical values [Fake = 0; Real = 1]
lab_enc = LabelEncoder()

#Applying to the dataset
data['label'] = lab_enc.fit_transform(data['label'])

In [ ]:
data

,texts,label
0,Q4: 2021-03-18 Earnings Summary,1
1,EPS of $0.97 beats by $0.12 | Revenue of $515....,0
2,"Good afternoon, and welcome to Ollie's Bargain...",1
3,Thank you. And good afternoon everyone. A pres...,1
4,Factors that might affect future results may n...,1
...,...,...
2660,So now let's go to the slide. So this gives yo...,1
2661,"And more importantly, 65% of our total sales a...",1
2662,Our exclusive sales participation reached 35% ...,1
2663,Because our data analysis shows that an averag...,1


In [ ]:
#Splitting the data into test and train
x_train,x_test,y_train,y_test = train_test_split(data['texts'], data.label, test_size=0.1, random_state=42)

In [ ]:
x_train
x_test

1419    Yes, we have. First of all, with regards to th...
2604    The 2019 results include a net gain on asset s...
478     Corporate services revenue were up 2% with QX ...
1264    Well, I will tell you most of the $350 million...
1741    Perfect. Thank you, and congrats again on the ...
                              ...                        
890     A recent technology milestone has been the com...
2504    Yeah, so we do anticipate in the coming months...
874     Ladies and gentlemen, thank you for standing b...
1271    Yes, let me also jump in on that. I couldn't w...
199     Secondly, KA clients have shifted more of thei...
Name: texts, Length: 267, dtype: object

In [ ]:
#Loading the tensorflow pretrained model
Embed = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
Trainable_Module = True
hub_layer = hub.KerasLayer(Embed, input_shape=[], dtype=tf.string, trainable=Trainable_Module)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)           #pre-trained text embedding layer
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 400,933
Trainable params: 400,933
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy']
              )

In [ ]:
EPOCHS = 50           #feel free to change this
BATCH_SIZE = 128      #feel free to change this

history = model.fit(x_train,y_train, batch_size = BATCH_SIZE,
                    epochs = EPOCHS, validation_split= 0.1,
                    verbose=1)

Epoch 1/50
17/17 [==============================] - 4s 31ms/step - loss: 0.7637 - accuracy: 0.2372 - val_loss: 0.4913 - val_accuracy: 0.6500
Epoch 2/50
17/17 [==============================] - 0s 8ms/step - loss: 0.4740 - accuracy: 0.7347 - val_loss: 0.3743 - val_accuracy: 0.8833
Epoch 3/50
17/17 [==============================] - 0s 8ms/step - loss: 0.4125 - accuracy: 0.8455 - val_loss: 0.3385 - val_accuracy: 0.8917
Epoch 4/50
17/17 [==============================] - 0s 8ms/step - loss: 0.3778 - accuracy: 0.8627 - val_loss: 0.3212 - val_accuracy: 0.8917
Epoch 5/50
17/17 [==============================] - 0s 8ms/step - loss: 0.3680 - accuracy: 0.8574 - val_loss: 0.3096 - val_accuracy: 0.8917
Epoch 6/50
17/17 [==============================] - 0s 8ms/step - loss: 0.3727 - accuracy: 0.8526 - val_loss: 0.2973 - val_accuracy: 0.8958
Epoch 7/50
17/17 [==============================] - 0s 8ms/step - loss: 0.3578 - accuracy: 0.8550 - val_loss: 0.2874 - val_accuracy: 0.9000
Epoch 8/50
17/17 [=

In [ ]:
accr = model.evaluate(x_test,y_test, verbose=0)
#print("Model Accuracy on Original Title Text: ",'{:.2%}'.format(accr[1]))

#Cleaned Title Text
# accr_c = model_c.evaluate(xc_test,yc_test, verbose=0)
#print("Model Accuracy on Cleaned Title Text: ",'{:.2%}'.format(accr_c[1]))



tab_data = [ [ json.[""], '{:.2%}'.format(accr[0]), '{:.2%}'.format(accr[1]) ] ]


tabulate(tab_data, headers=['Text','Label'], tablefmt='pretty')




print(tabulate(tab_data, headers=['','LOSS','ACCURACY'], tablefmt='pretty'))

+--------------------------------------+--------+----------+
|                                      |  LOSS  | ACCURACY |
+--------------------------------------+--------+----------+
| Model Trained on Original Title Text | 27.22% |  93.26%  |
+--------------------------------------+--------+----------+


In [ ]:
# Save
model_name = 'sentiment_dnn'
save_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/', f"{model_name}.keras")
model.save(save_path)
print(f"Saved keras pipeline model at {save_path}")

Saved keras pipeline model at /content/drive/MyDrive/Colab Notebooks/sentiment_dnn.keras


In [ ]:
model.predict(x_test)

array([[ 1.57476826e+01],
       [-1.55904317e+00],
       [ 1.23065100e+01],
       [ 2.86639977e+00],
       [ 1.07036667e+01],
       [ 2.09679947e+01],
       [ 1.44532900e+01],
       [ 8.56423569e+00],
       [-1.90702212e+00],
       [ 1.66992302e+01],
       [ 1.25851860e+01],
       [ 1.24221697e+01],
       [ 4.41454029e+00],
       [-4.53833723e+00],
       [ 9.50703526e+00],
       [ 1.11332817e+01],
       [ 6.21583843e+00],
       [ 1.33226833e+01],
       [ 1.73148994e+01],
       [ 9.34010983e+00],
       [ 9.20525932e+00],
       [ 1.45977707e+01],
       [ 2.96289444e+01],
       [ 7.99972439e+00],
       [ 3.65421057e+00],
       [ 6.24872744e-01],
       [ 1.14210119e+01],
       [-6.75359011e+00],
       [-4.38302231e+00],
       [ 2.00023098e+01],
       [ 1.50562706e+01],
       [ 1.04620161e+01],
       [ 1.50958967e+01],
       [ 1.88286972e+01],
       [ 8.54737186e+00],
       [ 1.60087452e+01],
       [ 1.55303402e+01],
       [ 8.76456451e+00],
       [ 7.9

In [ ]:
x_test[:1]

1419    Yes, we have. First of all, with regards to th...
Name: texts, dtype: object

In [ ]:
# 5 Randomly Sampled Records from Original Title Text
pred_df = pd.DataFrame({ 'Texts' : x_test.sample(n=10, random_state=1) })

In [ ]:
pred_df.reset_index(inplace=True,drop=True)


In [ ]:

# Function to convert numerical label to string
def label2str(x):
    if x == 0:
      return ["negative"]
    else:
      return ["positive"]

In [ ]:

# Add a new column with predictions
pred_df['ModelPred_OgTxt'] = [label2str(x) for x in model.predict_classes(pred_df.Texts)]
#pred_df['ModelPred_CleandTxt'] = [label2str(x) for x in model_c.predict_classes(pred_df.Random_News_Titles)]


In [ ]:
# Applying the highlight style to the prediction dataframe
pred_df

,Texts,ModelPred_OgTxt
0,Salarius Pharmaceuticals disclaims any obligat...,[positive]
1,But I can tell you that while this has been th...,[positive]
2,"So that if you take that away, the other is a ...",[negative]
3,"New QB subscriptions, however, increased to 22...",[positive]
4,So what I mean by that is they are doing proce...,[positive]
5,"Due to the pandemic, we've yet to benefit from...",[positive]
6,"Dr. O'Day is a pioneer in immuno-oncology, and...",[positive]
7,"Thank you, Jen. My decision to join the Agenus...",[positive]
8,A recent technology milestone has been the com...,[positive]
9,Okay. Thanks for clarifying that. And that mak...,[positive]


In [ ]:
model.predict_classes(['This is happy', 'This is horribly bad'])
#model.model.predict_classes(x_test)

array([[1],
       [0]], dtype=int32)

In [ ]:
[label2str(model.predict_classes([input])) for input in ['This is happy', 'This is horribly bad']]
#model.model.predict_classes(x_test)

[['positive'], ['negative']]

In [ ]:
print("Number of texts: ", data.shape[0])

Number of texts:  2665
